In [18]:
import sys
import os
print(os.environ.get('VIRTUAL_ENV'))
print(sys.version)
print(sys.executable)

/Users/hadare/Documents/CodingProjects/SBERT/sbert
3.10.14 (main, Mar 19 2024, 21:46:16) [Clang 15.0.0 (clang-1500.3.9.4)]
/Users/hadare/Documents/CodingProjects/SBERT/sbert/bin/python


In [19]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset

In [20]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    print("using MPS")


using MPS


In [21]:
# dataset = load_dataset('csv', './data/train.csv')
df_train = pd.read_csv('data/test.csv')
df_valid = pd.read_csv('data/dev.csv')
df_test = pd.read_csv('data/test.csv')
df_train.head()
df_valid.head()
df_test.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,translation1,sentence2,translation2
0,['neutral'],fiction,neutral,128144n,128144,She smiled back.,היא חייכה בחזרה.,She was so happy she couldn't stop smiling.,היא היתה כל כך שמחה שהיא לא יכלה להפסיק לחייך.
1,['neutral'],slate,neutral,145721n,145721,But such a show would have meant the museum ta...,אבל מופע כזה היה אומר שהמוזיאון יסתכל טוב על ה...,The museum didn't want to look too closely int...,המוזיאון לא רצה להתבונן מקרוב מדי בחלקו בעולם ...
2,['entailment'],travel,entailment,79085e,79085,In June you can watch dealers haggling over he...,בחודש יוני אפשר לצפות בסוחרים מתמקחים על ערמות...,During the summer months dealers barter for pi...,בחודשי הקיץ סוחרים סוחרים בערימות של פקעות תול...
3,['contradiction'],fiction,contradiction,133206c,133206,"119 Cynthia was back from the hospital, and I ...","סינתיה חזרה מבית החולים, ואני הצבתי את הכיסא ש...","Poirot talked to Cynthia himself, asking to vi...","פוארו דיבר עם סינתיה בעצמו, וביקש לבקר אותה בע..."
4,['contradiction'],slate,contradiction,107785c,107785,"Oh well, if we're not destined for matrimony i...","ובכן, אם לא נועדנו לנישואין בחיים האלה, אולי ה...","If we are married, I will buy you a beer or a ...","אם נתחתן, אקנה לך בירה או ד""ר פפר."


In [22]:
dataset_train = Dataset.from_pandas(df_train)
dataset_valid = Dataset.from_pandas(df_valid)
dataset_test = Dataset.from_pandas(df_test)

In [23]:

# Define a function to process the dataset
def preprocess_data(data):
    premise = data['translation1']
    hypothesis = data['translation2']
    label = data['gold_label']
    
    # Convert labels to numerical values if needed
    label_mapping = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
    label = label_mapping.get(label, -1)  # -1 for any unknown labels

    return premise, hypothesis, label

# Apply preprocessing to the train, validation, and test datasets
train_data = [preprocess_data(row) for row in dataset_train]
valid_data = [preprocess_data(row) for row in dataset_valid]
test_data = [preprocess_data(row) for row in dataset_test]

# Convert to pandas DataFrame for easier manipulation
train_df = pd.DataFrame(train_data, columns=['premise', 'hypothesis', 'label'])
valid_df = pd.DataFrame(valid_data, columns=['premise', 'hypothesis', 'label'])
test_df = pd.DataFrame(test_data, columns=['premise', 'hypothesis', 'label'])

# Save processed data if needed
# train_df.to_csv('train.csv', index=False)
# valid_df.to_csv('valid.csv', index=False)
# test_df.to_csv('test.csv', index=False)


In [24]:
print(df_train.isnull().sum().sum())
print(df_valid.isnull().sum().sum())
print(df_test.isnull().sum().sum())

0
0
0


In [25]:
# columns_to_lower = ['sentence1', 'sentence2']
# df_train[columns_to_lower] = df_train[columns_to_lower].map(lambda x: x.lower())
# df_train.head()

In [26]:
# from transformers import AutoTokenizer, BertTokenizer

# tokenizer_aleph = AutoTokenizer.from_pretrained('onlplab/alephbert-base')
# tokenizer_mbert = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


In [27]:
# print(tokenizer_aleph)
# tokenizer_mbert

In [28]:
from sentence_transformers import SentenceTransformer, SentencesDataset, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader

# Load the base model
model_name = 'onlplab/alephbert-base'  # e.g., 'onlplab/alephbert-base' 
# model_name = 'google-bert/bert-base-multilingual-cased'
# model_name = 'dicta-il/dictabert'
model = SentenceTransformer(model_name)


No sentence-transformers model found with name dicta-il/dictabert. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dicta-il/dictabert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

In [12]:
# Prepare InputExamples
train_examples = [InputExample(texts=[row['premise'], row['hypothesis']], label=row['label']) for _, row in train_df.iterrows()]
valid_examples = [InputExample(texts=[row['premise'], row['hypothesis']], label=row['label']) for _, row in valid_df.iterrows()]

# Create datasets and dataloaders
train_dataset = SentencesDataset(train_examples, model)
valid_dataset = SentencesDataset(valid_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
valid_dataloader = DataLoader(valid_dataset, shuffle=False, batch_size=32)


In [13]:
# Define loss function
train_loss = losses.MultipleNegativesRankingLoss(model)

# Train the model
num_epochs = 4
warmup_steps = 100  # Adjust based on dataset size

# valid_examples = []

# # Extract examples from the DataLoader
# for batch in valid_dataloader:
#     sentences1, sentences2, scores = batch
#     for sent1, sent2, score in zip(sentences1, sentences2, scores):
#         sent1 = sent1 if isinstance(sent1, str) else sent1.item()
#         sent2 = sent2 if isinstance(sent2, str) else sent2.item()
#         score = score.item()
#         valid_examples.append(InputExample(texts=[sent1, sent2], label=score))

# # Create the evaluator
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_examples, name='dev')

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    # evaluator=evaluator,
    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_examples, name='dev'),
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path='output/sbert_hebrew_model'
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/628 [00:00<?, ?it/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

{'loss': 0.2226, 'grad_norm': 1.0029336214065552, 'learning_rate': 4.732824427480917e-06, 'epoch': 3.18}
{'train_runtime': 470.2055, 'train_samples_per_second': 42.535, 'train_steps_per_second': 1.336, 'train_loss': 0.17946006139372564, 'epoch': 4.0}


In [15]:
# Save the model
model.save_pretrained('./trained_sbert_model')
# tokenizer.save_pretrained('./trained_sbert_model')


In [16]:
# Load the Hebrew STS benchmark dataset (replace with actual data loading)
# sts_data = load_dataset('./data/heb_sts_test.csv')
sts_data = load_dataset('csv', data_files='./data/heb_sts_test.csv')
print(sts_data)

# Prepare the evaluation dataset
sts_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['score']) for row in sts_data]

# Evaluate
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(sts_examples, name='hebrew-sts')
model.evaluate(evaluator)


DatasetDict({
    train: Dataset({
        features: ['sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})


TypeError: string indices must be integers

In [17]:
from datasets import load_dataset
from sentence_transformers import InputExample
import pandas as pd

# Load the dataset
sts_data = load_dataset('csv', data_files='./data/heb_sts_test.csv')

# Access the dataset
dataset = sts_data['train']  # or the appropriate split name

# Convert to DataFrame for inspection
df = pd.DataFrame(dataset)

# Prepare the evaluation dataset
sts_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['score']) 
                for row in df.to_dict(orient='records')]

# Initialize the evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(sts_examples, name='hebrew-sts')

# Assuming you have a trained model
model.evaluate(evaluator)


print("Evaluation results for the model on Hebrew STS benchmark:")
print(evaluator(model, output_path='output/sbert_hebrew_model'))


Evaluation results for the model on Hebrew STS benchmark:
{'hebrew-sts_pearson_cosine': 0.7116431915772918, 'hebrew-sts_spearman_cosine': 0.6906499377642947, 'hebrew-sts_pearson_manhattan': 0.7160632824099897, 'hebrew-sts_spearman_manhattan': 0.6995660820383547, 'hebrew-sts_pearson_euclidean': 0.7167087784665823, 'hebrew-sts_spearman_euclidean': 0.7002192924941905, 'hebrew-sts_pearson_dot': 0.6435424677387844, 'hebrew-sts_spearman_dot': 0.6197261353452318, 'hebrew-sts_pearson_max': 0.7167087784665823, 'hebrew-sts_spearman_max': 0.7002192924941905}
